In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import transformers
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

from transformers import RobertaTokenizer, AutoModel

import sys
sys.path.append('/content/drive/MyDrive/skripsi')
!ls /content/drive/MyDrive/skripsi

check_lang.py	     finetuning_RoBERTa2.ipynb	Preprocessing_data_sinta.ipynb	src
cluster_sinta.ipynb  finetuning_RoBERTa.ipynb	Preprocessing.py
data		     model			__pycache__


In [ ]:
from Preprocessing import preprocess_text

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/skripsi/data/data_sinta_cleaned2.csv')
df

,jid,desc,title,date,journal,eissn
0,8,Belakangan ini Lembaga PAUD berbondong-bondong...,kemampuan bahasa inggris awal pada periode lin...,2023-10-05,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
1,8,The assessment process carried out in schools ...,strengthening early childhood learning outcome...,2022-06-09,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
2,8,Kewajiban guru PAUD untuk memberikan layanan b...,eksplorasi deskriptif tentang layanan bimbinga...,2023-06-07,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
3,8,Pandemi Covid-19 merambah sejak awal tahun 202...,perspektif orang tua terhadap pelaksanaan les ...,2021-06-02,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
4,8,Dalam upaya mencapai Sustainable Development G...,strategi pencegahan dan penanganan stunting mu...,2023-12-26,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
...,...,...,...,...,...,...
9860,2,The level of understanding of accounting to be...,pengaruh kecerdasan emosional kecerdasan intel...,2017-08-06,E-JURNAL AKUNTANSI,2302-8556
9861,2,The purpose of this study was to examine the i...,pengaruh ukuran perusahaan kompleksitas operas...,2017-02-15,E-JURNAL AKUNTANSI,2302-8556
9862,2,The purpose of this study is to determine the ...,kinerja dinas pariwisata bali berdasarkan kons...,2016-11-10,E-JURNAL AKUNTANSI,2302-8556
9863,2,Timeliness of financial statements is an impor...,determinan audit delay dan pengaruhnya pada ha...,2015-05-25,E-JURNAL AKUNTANSI,2302-8556


In [ ]:
df.jid = df.jid.apply(lambda x : x - 1)

In [ ]:
df.dropna(inplace=True)
df = df[df.desc != 'KOSONG']
df = df[df.title != 'KOSONG']
df

,jid,desc,title,date,journal,eissn
0,7,Belakangan ini Lembaga PAUD berbondong-bondong...,kemampuan bahasa inggris awal pada periode lin...,2023-10-05,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
1,7,The assessment process carried out in schools ...,strengthening early childhood learning outcome...,2022-06-09,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
2,7,Kewajiban guru PAUD untuk memberikan layanan b...,eksplorasi deskriptif tentang layanan bimbinga...,2023-06-07,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
3,7,Pandemi Covid-19 merambah sejak awal tahun 202...,perspektif orang tua terhadap pelaksanaan les ...,2021-06-02,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
4,7,Dalam upaya mencapai Sustainable Development G...,strategi pencegahan dan penanganan stunting mu...,2023-12-26,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,2549-8959
...,...,...,...,...,...,...
9860,1,The level of understanding of accounting to be...,pengaruh kecerdasan emosional kecerdasan intel...,2017-08-06,E-JURNAL AKUNTANSI,2302-8556
9861,1,The purpose of this study was to examine the i...,pengaruh ukuran perusahaan kompleksitas operas...,2017-02-15,E-JURNAL AKUNTANSI,2302-8556
9862,1,The purpose of this study is to determine the ...,kinerja dinas pariwisata bali berdasarkan kons...,2016-11-10,E-JURNAL AKUNTANSI,2302-8556
9863,1,Timeliness of financial statements is an impor...,determinan audit delay dan pengaruhnya pada ha...,2015-05-25,E-JURNAL AKUNTANSI,2302-8556


In [ ]:
# Define Dataset class
class RobertaDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = 256

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]['desc']
        labels = self.data.iloc[index][['jid']].values.astype(int)
        encoding = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=self.max_length)
        input_ids = encoding['input_ids'][0]
        attention_mask = encoding['attention_mask'][0]
        input_ids = nn.functional.pad(input_ids, (0, self.max_length - input_ids.shape[0]), value=0)
        attention_mask = nn.functional.pad(attention_mask, (0, self.max_length - attention_mask.shape[0]), value=0)
        return input_ids, attention_mask, torch.Tensor(labels)

In [ ]:
# Load tokenizer and model
model_checkpoint = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset = RobertaDataset(df, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class RobertaClassifier(nn.Module):
    def __init__(self, num_labels):
        super(RobertaClassifier, self).__init__()
        self.roberta = AutoModel.from_pretrained(model_checkpoint)
        self.classifier = nn.Sequential(
            nn.Linear(self.roberta.config.hidden_size, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs['last_hidden_state'][:, 0, :]
        x = self.classifier(x)
        return x

In [ ]:
num_labels = 12
model = RobertaClassifier(num_labels).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

num_epochs = 2
n_total_steps = len(train_loader)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for epoch in range(num_epochs):
    for i, batch in enumerate(train_loader):
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.view(-1)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels.long())
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch + 1}/{num_epochs}, batch {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

epoch 1/2, batch 100/309, loss = 1.8993
epoch 1/2, batch 200/309, loss = 1.6956
epoch 1/2, batch 300/309, loss = 1.0555
epoch 2/2, batch 100/309, loss = 0.8329
epoch 2/2, batch 200/309, loss = 0.6172
epoch 2/2, batch 300/309, loss = 0.5868


In [ ]:
# Create the directory if it doesn't exist
os.makedirs('model', exist_ok=True)
# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/skripsi/model/finetuned_model_roberta2.pt')